In [1]:
import itertools
import numpy as np
import pandas as pd
from fbprophet.diagnostics import cross_validation
from fbprophet.diagnostics import performance_metrics
from fbprophet import Prophet

In [2]:
data= pd.read_csv("./data/validations-sur-le-reseau-ferre-nombre-de-validations-par-jour-1er-sem.csv", sep=";")
data.JOUR = pd.to_datetime(data.JOUR, errors='coerce')#formater les date en datetime
data = data.sort_values(by=["JOUR"])# trier nos données

In [3]:
#on suppose que le nombre de validations Moins de x est égal à x
data["NB_VALD"] = data["NB_VALD"].apply(lambda column : int(column.replace("Moins de ", "")))

In [4]:
data_stat1 = data[data["LIBELLE_ARRET"] == "OLYMPIADES"].groupby(["JOUR", "CATEGORIE_TITRE"]).agg({'NB_VALD':'sum'})
data_stat1.reset_index(inplace=True)
data_stat1

,JOUR,CATEGORIE_TITRE,NB_VALD
0,2019-01-01,?,53
1,2019-01-01,AMETHYSTE,210
2,2019-01-01,FGT,229
3,2019-01-01,IMAGINE R,660
4,2019-01-01,NAVIGO,2374
...,...,...,...
1465,2019-06-30,IMAGINE R,1479
1466,2019-06-30,NAVIGO,5694
1467,2019-06-30,NAVIGO JOUR,5
1468,2019-06-30,NON DEFINI,5


In [5]:
data_prev =  data_stat1.groupby(["JOUR"]).agg({'NB_VALD':'sum'})
data_prev.reset_index(inplace=True)
data_prev = data_prev.rename(columns={"JOUR": "ds", "NB_VALD": "y"})

In [6]:
data_prev

,ds,y
0,2019-01-01,3764
1,2019-01-02,14074
2,2019-01-03,15902
3,2019-01-04,16644
4,2019-01-05,11062
...,...,...
176,2019-06-26,19909
177,2019-06-27,20034
178,2019-06-28,20415
179,2019-06-29,12149


In [8]:
param_grid = {  
    'changepoint_prior_scale': [0.001, 0.01, 0.1, 0.5],
    'seasonality_prior_scale': [0.01, 0.1, 1.0, 10.0],
    #'holidays_prior_scale' : [0.01, 0.1, 1.0, 10.0],
    #'seasonality_mode' : ['additive', 'multiplicative']
    
}

# Generate all combinations of parameters
all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
rmses = []  # Store the RMSEs for each params here

# Use cross validation to evaluate all parameters
for params in all_params:
    m = Prophet(**params).fit(data_prev)  # Fit model with given params
    df_cv = cross_validation(m, horizon='30 days', parallel="processes")
    df_p = performance_metrics(df_cv, rolling_window=1)
    rmses.append(df_p['rmse'].values[0])

# Find the best parameters
tuning_results = pd.DataFrame(all_params)
tuning_results['rmse'] = rmses
print(tuning_results)


INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Making 5 forecasts with cutoffs between 2019-04-01 00:00:00 and 2019-05-31 00:00:00
INFO:fbprophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7fa72028fcf8>
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Making 5 forecasts with cutoffs between 2019-04-01 00:00:00 and 2019-05-31 00:00:00
INFO:fbprophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7fa72028fbe0>
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet 

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Making 5 forecasts with cutoffs between 2019-04-01 00:00:00 and 2019-05-31 00:00:00
INFO:fbprophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7fa7202afe80>
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Making 5 forecasts with cutoffs between 2019-04-01 00:00:00 and 2019-05-31 00:00:00
INFO:fbprophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7fa72020acf8>


    changepoint_prior_scale  seasonality_prior_scale         rmse
0                     0.001                     0.01  5607.968796
1                     0.001                     0.10  4334.362381
2                     0.001                     1.00  4266.011889
3                     0.001                    10.00  4258.719827
4                     0.010                     0.01  7594.064355
5                     0.010                     0.10  4408.567854
6                     0.010                     1.00  4340.946104
7                     0.010                    10.00  4203.496942
8                     0.100                     0.01  5634.118889
9                     0.100                     0.10  3941.338136
10                    0.100                     1.00  4024.692370
11                    0.100                    10.00  3980.191735
12                    0.500                     0.01  5516.166650
13                    0.500                     0.10  5140.989487
14        